In [ ]:
########################################################################
#################### Carga de módulos de Python ########################
########################################################################

import pandas as pd
import numpy as np
import os
from plotnine import *
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

In [ ]:
########################################################################
############ Lectura de archivo y preprocesamiento de datos ############
########################################################################

mi_tabla = pd.read_csv("mpg.csv")

dataset = mi_tabla.select_dtypes(np.number)
dataset.columns

In [ ]:
###### Supongamos que la columna objetivo se llama Co y las variables independientes se llaman C1,C2,...,Cn.

## Variable objetivo
nombre_variable_objetivo = "mpg"
objetivo = dataset[nombre_variable_objetivo]

## Variable(s) independiente(s)
variables_independientes = dataset[["horsepower","weight","displacement","aceleration"]]
nombre_variables_independientes = variables_independientes.columns
nombre_variables_independientes

In [ ]:
########################################################################
########################   Ajuste del modelo   #########################
########################################################################

####### División en entrenamiento y prueba con 20% de prueba
X_train, X_test, y_train, y_test = train_test_split(variables_independientes, objetivo, test_size=0.2,random_state=42)

# Construye el pipeline
pipeline = Pipeline([
    ("imputacion", SimpleImputer(strategy="mean")),
    ("modelo", LinearRegression())
])

# Entrena el modelo con X_train y y_train
pipeline.fit(X_train, y_train)

# Extrae el modelo entrenado (etapa final del pipeline)
mi_modelo = pipeline.named_steps["modelo"]

coeficientes = pd.DataFrame({
    "Variable": ["Intercepto"] + list(variables_independientes.columns),
    "Coeficiente": [mi_modelo.intercept_] + list(mi_modelo.coef_)
})

print(coeficientes)

In [ ]:
########################################################################
##########################   Modelo final   ############################
########################################################################

texto_modelo = f"{nombre_variable_objetivo} predicho = {coeficientes.Coeficiente[0]:.3f}"

for x in range(1,coeficientes.shape[0]):
    signo = " + " if coeficientes.Coeficiente[x] >= 0 else " - "
    texto_modelo = texto_modelo + f"{signo}{np.abs(coeficientes.Coeficiente[x]):.3f} {coeficientes.Variable[x]}"

print(f"El modelo es\n\n {texto_modelo}")

In [ ]:
datos_reales = dataset[[nombre_variables_independientes[0],nombre_variable_objetivo]]

(ggplot(data=datos_reales) +
    geom_point(mapping=aes(x=datos_reales.columns[0],y=datos_reales.columns[1]),color="blue") +
    geom_smooth(mapping=aes(x=datos_reales.columns[0],y=datos_reales.columns[1]),method="lm",color="red",se=False)
)

In [ ]:
########################################################################
####################### Evaluación del modelo ##########################
########################################################################

from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

def calcular_metricas(y_real, y_pred, n, p):
    """
    Calcula MAE, MSE, RMSE, R2 y R2 ajustado.

    Parámetros:
    - y_real: valores reales
    - y_pred: valores predichos
    - n: número de observaciones
    - p: número de predictores (sin incluir el intercepto)
    """
    mae = mean_absolute_error(y_real, y_pred)
    mse = mean_squared_error(y_real, y_pred)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_real, y_pred)
    r2_ajustado = 1 - (1 - r2) * (n - 1) / (n - p - 1)

    return pd.Series({
        "MAE": mae,
        "MSE": mse,
        "RMSE": rmse,
        "R²": r2,
        "R² ajustado": r2_ajustado
    })


# Predicciones en entrenamiento
y_train_pred = pipeline.predict(X_train)
metricas_entrenamiento = calcular_metricas(y_train, y_train_pred, n=len(y_train), p=X_train.shape[1])

# Predicciones en prueba
y_test_pred = pipeline.predict(X_test)
metricas_prueba = calcular_metricas(y_test, y_test_pred, n=len(y_test), p=X_test.shape[1])

# Mostrar resultados
metricas_df = pd.DataFrame({
    "Entrenamiento": metricas_entrenamiento,
    "Prueba": metricas_prueba
})

metricas_df

In [ ]:
### Supongamos que queremos hallar las predicciones en la tabla mis_nuevos_datos.csv

columnas_entrenamiento = list(variables_independientes.columns)
nuevos_datos = pd.read_csv("mpg_desconocidos.csv")
nuevos_datos[columnas_entrenamiento]

In [ ]:
def predecir_nuevos_datos(modelo_pipeline, nuevos_datos, columnas_entrenamiento):
    """
    Realiza predicciones sobre nuevos datos usando un modelo entrenado.

    Parámetros:
    - modelo_pipeline: objeto de modelo entrenado.
    - nuevos_datos: DataFrame con los nuevos casos a predecir.
    - columnas_entrenamiento: lista con los nombres de columnas usadas para entrenar el modelo.

    Devuelve:
    - Un array con las predicciones.
    """
    # Verifica si faltan columnas
    columnas_faltantes = set(columnas_entrenamiento) - set(nuevos_datos.columns)
    if columnas_faltantes:
        raise ValueError(f"Faltan columnas necesarias: {columnas_faltantes}")

    # Verifica si hay columnas adicionales
    columnas_adicionales = set(nuevos_datos.columns) - set(columnas_entrenamiento)
    if columnas_adicionales:
        print(f"Advertencia: se ignorarán columnas adicionales: {columnas_adicionales}")

    # Reordenar columnas para que coincidan con el modelo
    nuevos_datos_ordenado = nuevos_datos[columnas_entrenamiento]

    # Realizar predicción
    return modelo_pipeline.predict(nuevos_datos_ordenado)

predicciones_nuevos_datos = predecir_nuevos_datos(pipeline, nuevos_datos, columnas_entrenamiento)
predicciones_df = pd.DataFrame(predicciones_nuevos_datos, columns=["Predicción"])
nuevos_con_prediccion = pd.concat([nuevos_datos.reset_index(drop=True), predicciones_df], axis=1)

# Mostrar resultado
nuevos_con_prediccion